# Kaggle - Titanic Survival Passengers Prediction 

This is my first Kaggle competition - <a herf="https://www.kaggle.com/c/titanic"> predict the survival passengers from the Titanic</a>. In this project, I have learned 
- Data processing with **numpy and pandas** 
    - understand different types of data 
    - work with missing values 
    - feature engineering 
- Data visiualization with **seaborn**
- Data prediction with **sklearn** 
    - decision tree 
    - kNN 
    - random forest
    - logistic regression
    - cross validation to determine the optimal approach 

The accuracy of my model is **85%** on dev set and **78%** on the final prediction set (<a herf=https://www.kaggle.com/jieyu7/competitions>check out my kaggle page</a>). Though I do find that simpler and easier feature engineering process gives a model with a higher accuracy. But the main goal of this project is to learn different techniques of data analysis. I will learn about the efficiency of different feature engineering methods in the next kaggle project. 

2020.05.31

        

# Load data 

In [1]:
## imports
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings('ignore')
import os
from matplotlib import pyplot as plt

In [13]:
## load data 
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Understand the data 

In [14]:
print(train_df.info())
print('*'*50)
print(test_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB
None
**************************************************
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  

## Remove unrelated columns 

The first step is to look at the data and remove any unrelated columns. Here only the `PassengerId` is unrelated and has no influence on the prediction.

`PassengerId` is removed from both the training and the test sets but saved as a separate column for creating the submission file

In [15]:
PassengerId = test_df['PassengerId']

train_df.drop(['PassengerId'], axis = 1, inplace = True)
test_df.drop(['PassengerId'], axis = 1, inplace = True)

## Group features 

The second step is to find out the data type of each feature, and whether each feature has unique values or missing values. The following is a brief summary. This summary is a guidance of the following data processing steps. This guidance could be changed as my understanding of the data set gets deeper. 

**bold** : missing values

*italic* : unique values

- numertical features (shown in numbers)
    1. Pclass
    2. **Age**
    3. SibSp
    4. Parch
    5. **Fare** 
- categorical features (shown in letters) : convert to numbers and get dummies 
    1. Sex
    2. **Embarked**
- alphanumerical/text features (shown in both letters and numbers) : engineering features 
    1. **Cabin**
    2. *Ticket*
    3. *Name*

    


# Process the data 

To process the data, I start with the simpler tasks and continue with harder tasks. The steps are 
1. convert categorical values to numbers
2. work with missing values 
3. work with feature engineering 
4. convert categorical values to dummies 

I choose not to convert the categorical values to dummies at first because this would create too many columns that makes the visualization complex. Before working on the data. I combine the `train_df` and `test_df` together as `all_data`

In [16]:
train_df.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [17]:
all_data = pd.concat([train_df,test_df], ignore_index=False)

## Convert categorical features to numbers 
There are two categorical features: `Sex` and `Embarked`. 

In [18]:
print(all_data['Sex'].unique())
print(all_data['Embarked'].unique())

['male' 'female']
['S' 'C' 'Q' nan]


Because `Embarked` has missing values, we need to work with the `nan` first and them factorize it. So in this step. only `Sex` is converted to numberical values where `male` is `0` and `female` is `1`. Both `train_df` and `test_df` are included. 

In [ ]:
all_data['Sex'] = pd.factorize(all_data['Sex'])[0] 

## Work with missing values 

I use a simple function to print the percentage of missing values in each feature. This function is from another kaggle post that I cannot find now. 

In [ ]:

def missing_percentage(df):
    """
        This function takes a DataFrame(df) as input and returns two columns, 
        total missing values and total missing values percentage
    """
    total = df.isnull().sum().sort_values(ascending = False)
    percent = round(df.isnull().sum().sort_values(ascending = False)/len(df)*100,2)
    return pd.concat([total, percent], axis=1, keys=['Total','Percent'])

In [ ]:
missing_percentage(all_data)

In [ ]:
missing_percentage(train_df)

In [ ]:
missing_percentage(test_df)

Both in `test_df` and in `train_df`, features `Cabin` and `Age` have missing values. 2 missing value in `Embarked` and 1 missing value in `Fare`. I will first look into `Fare` and `Embarked`. For the other two features, because there are so many missing values, it is highly likely there will be feature engineering involved. 


### Embarked 
The following is another function from the same kaggle post mentioned above. 

In [ ]:
def percent_value_counts(df, feature):
    """This function takes in a dataframe and a column and finds the percentage of the value_counts"""
    percent = pd.DataFrame(round(df.loc[:,feature].value_counts(dropna=False, normalize=True)*100,2))
    ## creating a df with th
    total = pd.DataFrame(df.loc[:,feature].value_counts(dropna=False))
    ## concating percent and total dataframe

    total.columns = ["Total"]
    percent.columns = ['Percent']
    return pd.concat([total, percent], axis = 1)
    

In [ ]:
percent_value_counts(train_df, 'Embarked')

Because the majority of `Embarked` is `S`, it is reasonable to fill the missing values as `S`. But I will plot the relation between `Embarked` and `Fare` and `Pclass` to see if `S` is reasonable. Because I suspect the `Embarked` is related to these two features.

In [ ]:
all_data[all_data.Embarked.isnull()]

For the missing vales, they are both female, in `Pclass`1 and `Fare` of 80 ; I use seaborn plots to see if `Pclass` and `Fare` are related to `Embarked` 

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style('darkgrid')
fig, ax = plt.subplots(figsize=(7,5))
ax = sns.boxplot(x="Embarked", y="Fare", hue="Pclass", data=all_data);



with `Pclass` = 1 and `Fare` = 80, it is more likely to embark at C. Therefore, I will fill the missing values with C 

In [ ]:
all_data['Embarked'].fillna("C", inplace=True)

After filling the missing values, this feature is again factorized. 

In [ ]:
all_data['Embarked'] = pd.factorize(all_data['Embarked'])[0]

### Fare

there is only one missing value of fare in test_df
see how it is like

In [ ]:
all_data[all_data.Fare.isnull()]

`Pclass` = 3 and `Embarked` at 0, from the above figure we can roughly estimate the fare as the mean

In [ ]:
## a simpler version 

# missing_value_fare = all_data[(all_data['Pclass'] == 3) & (all_data['Embarked'] == 0)].Fare.mean()
# ## replace the test.fare null values with test.fare mean
# all_data.Fare.fillna(missing_value_fare, inplace=True)

## Engineering feature 

Features need to be engineered

1. Cabin: a letter + some number; the letter could be used 
2. Name: mainly use the titles of the names 
3. Ticket: see if there is anything we can use 
4. Age: missing value with ML; see if needed to group them 

Features could be engineered
1. SibSp
2. parch 
<div>
some ideas

    - put into different groups rather than continuous values 
    - put together as family size 

### Cabin 

The naming convention of the `Cabin` is a letter followed by some numbers. People with multiple cabins still have the same letter. So we can just use the first letter as the feature. The `Cabin` is directly associated with class, according to an answer from <a herf="https://www.quora.com/What-were-the-ticket-prices-to-board-the-Titanic">Quora</a>. Therefore, the missing values in `Cabin` can be determined by its fare and class. 

In [ ]:
## cabin with values 
all_cabin_nonan = all_data[all_data['Cabin'].notnull()]

all_cabin_nonan['Cabin'] = [i[0] for i in all_cabin_nonan['Cabin']]

all_cabin_nonan = all_cabin_nonan.sort_values(by='Cabin')

sns.catplot(x="Cabin", y="Fare", hue='Pclass', kind="swarm", data=all_cabin_nonan);

From the plot, in each kind, the `Fare` has a large range. To better predict the missing values, here I decide to use bootstrap to generate values. For each `Fare` range and `Pclass` group, the missing `Cabin` value is randomly selected from the known distribution. For example, For `Pclass` = 1 and `Fare` between 200-300, the missing `Cabin` value is chosen from the distribution of values that satisfy this condition. Therefore, I need to decide the group criterion. 

pseudo code:

`if Pclass == 3:
    if Fare > 20:  # this threshold is determined in the following code 
        Cabin = F 
    else: Cabin = random.select from dist of Cabin == E, F, G & Fare < 20 and  Pclass == 3
 if Pclass == 2:
     
    



`
        

In [ ]:
## find the threshold from F column
all_cabin_nonan[["Fare","Pclass"]][(all_cabin_nonan["Cabin"]=='F') & (all_cabin_nonan["Pclass"]==3)].max()

In [ ]:
all_cabin_nonan["Cabin"] = pd.factorize(all_cabin_nonan["Cabin"])[0]

In [ ]:
all_cabin_nonan["Cabin"].unique()

In [ ]:
all_cabin_nan = all_data[all_data['Cabin'].isnull()]

In [ ]:
cabin=np.zeros((all_data['Cabin'].isnull().sum(),1))
all_cabin_pclass3 = all_cabin_nonan[all_cabin_nonan["Pclass"] == 3]
all_cabin_pclass2 = all_cabin_nonan[all_cabin_nonan["Pclass"] == 2]
all_cabin_pclass1 = all_cabin_nonan[all_cabin_nonan["Pclass"] == 1]

for i in range(len(cabin)):
    data = all_cabin_nan.iloc[i]
    if data['Pclass'] == 3:
        if data['Fare'] > 20:
            cabin[i] = 5
        else:
            group = all_cabin_pclass3[all_cabin_pclass3['Fare'] <= 20]
            dist = group['Cabin'][(group["Cabin"] >= 3) & (group["Cabin"] <= 5)]
            cabin[i] = np.random.choice(dist)
    elif data['Pclass'] == 2:
        if data['Fare'] > 20:
            cabin[i] = 5
        else:
            group = all_cabin_pclass2[all_cabin_pclass2['Fare'] <= 20]
            dist = group['Cabin'][(group["Cabin"] >= 2) & (group["Cabin"] <= 5)]
            cabin[i] = np.random.choice(dist)
    else:
        if data['Fare'] > 500:
            cabin[i] = 1
        elif data['Fare'] > 200:
            group = all_cabin_pclass1[all_cabin_pclass1['Fare'] > 200]
            dist = group['Cabin'][(group["Cabin"] >= 1) & (group["Cabin"] <= 2)]
            cabin[i] = np.random.choice(dist)
        elif data['Fare'] < 100:
            group = all_cabin_pclass1[all_cabin_pclass1['Fare'] < 100]
            dist = group['Cabin'][(group["Cabin"] >= 0) & (group["Cabin"] <= 4)]
            # cabin T is ignored because there is only one of it -> statistically very small
            cabin[i] = np.random.choice(dist)
        else:
            group = all_cabin_pclass1[(all_cabin_pclass1['Fare'] <= 200) & (all_cabin_pclass1['Fare'] >= 100)]
            dist = group['Cabin'][(group["Cabin"] >= 1) & (group["Cabin"] <= 4)]
            cabin[i] = np.random.choice(dist)

In [ ]:
all_cabin_nan['Cabin'] = cabin

all_data = pd.concat([all_cabin_nonan,all_cabin_nan], ignore_index=False) 

all_data['Cabin'] = all_data['Cabin'].astype(int)
# train_df_new = all_data_new[all_data_new['Survived'].notnull()]
# test_df_new = all_data_new[all_data_new['Survived'].isnull()]

In [ ]:
all_data.info()

### Name 

For the `Name`, I will extract the title as a new feature and get rid of the `Name` feature.

In [ ]:
all_data['Title']=all_data.Name.apply(lambda x: x.split('.')[0].split(',')[1].strip())
all_data['Title'].unique()

I initially found a method of creating the `Title` with 4 different categories: Mr, Miss, Mrs and Rare, where Rare includes all other different titles such as Dr. Then I realized that simply putting all other titles as `Rare` is ineffective and could impact the accuracy when predicting `Age` as `Age` appears to be associated with titles. Hence I found another way of creating the `Title` where it has more categories from this <a herf="https://medium.com/i-like-big-data-and-i-cannot-lie/how-i-scored-in-the-top-9-of-kaggles-titanic-machine-learning-challenge-243b5f45c8e9">post<a/>.


In [ ]:
## this is the first one. 
# all_data_new['Title'] = all_data_new['Title'].replace(['Lady', 'Countess','Capt', 'Col',\
#  	'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare') ## simply put all the rare titles in one group is not wise. because the title is important in the Age prediction later! 

# all_data_new['Title'] = all_data_new['Title'].replace('Mlle', 'Miss') # Mlle to Miss
# all_data_new['Title'] = all_data_new['Title'].replace('Ms', 'Miss') # Ms to Miss
# all_data_new['Title'] = all_data_new['Title'].replace('Mme', 'Mrs') # Mme to Mrs
    

In [ ]:
# this is the second one. 
newtitles={
    "Capt":       "Officer",
    "Col":        "Officer",
    "Major":      "Officer",
    "Jonkheer":   "Royalty",
    "Don":        "Royalty",
    "Sir" :       "Royalty",
    "Dr":         "Officer",
    "Rev":        "Officer",
    "the Countess":"Royalty",
    "Dona":       "Royalty",
    "Mme":        "Mrs",
    "Mlle":       "Miss",
    "Ms":         "Mrs",
    "Mr" :        "Mr",
    "Mrs" :       "Mrs",
    "Miss" :      "Miss",
    "Master" :    "Master",
    "Lady" :      "Royalty"}

In [ ]:
all_data['Title']=all_data.Title.map(newtitles)

In [ ]:
## Make sure there is no missing values 
all_data['Title'][all_data['Title'].isnull()]

In [ ]:
# drop the Name feature and factorize the Title feature 
all_data.drop(['Name'], axis=1, inplace=True)

all_data["Title"] = pd.factorize(all_data["Title"])[0]

### Ticket

I have not yet found an effective approach of dealing with `Ticket`, hence here I choose to drop it. I looked into some discussion posts on kaggle but I think they requires more energy than necessary. 

In [ ]:
all_data['Ticket'].describe()

In [ ]:
ticket = all_data['Ticket']

all_data.drop(['Ticket'], axis=1, inplace=True)

all_data.head()

### Age 

Because we need to use `Age` as a feature to predict survival rate, `Survival` is not used here to predict age. (not sure if this is correct. But if survival rate is used to predict age and then age is again used to predict survival, wouldn't it be a circulation?)

My first thought is to find out if there is any relation between other features and `Age` by plotting using seaborn. If the distribution of age significantly differs from the feature, this feature is included in the prediction of `Age`. <div>
Features included here are `Sex, Pclass, Cabin, Embarked, Title,` and `Fare`. `SibSp` and `Parch` are not included because there is no direct rational relation among these features. 

In [ ]:
features = ['Sex','Pclass','Cabin','Embarked','Title','Fare']

In [ ]:
i=0
fig, axes = plt.subplots(ncols=2,nrows=3, figsize=(20,12))
axes = axes.flatten()
for feature in features:
    if feature == 'Fare':
        sns.scatterplot(x='Age', y=feature, data=all_data,legend=False, ax = axes[i])
    else:
        y_axis = all_data[feature].unique()
        for y in y_axis:
            sns.kdeplot(all_data.loc[(all_data[feature] == y),'Age'] , shade=True,label= feature +" "+ str(y), ax = axes[i]);
    #     axes[i].title('Age Distribution '+ feature, fontsize = 25, pad = 40)
    #     axes[i].xlabel("Age", fontsize = 15, labelpad = 20)
    #     axes[i].ylabel('Frequency', fontsize = 15, labelpad= 20);
    i=i+1

From the plot we can see that features including `Pclass, Cabin`, and `Title`. 

However, after some tests, the random forest method is not working that well in the prediction. Then I decide to assign the mean of the `Title` feature to the corresponding missing value. This idea is inspired by this <a herf="https://www.kaggle.com/goldens/titanic-on-the-top-with-a-simple-model"> discussion<a/>.
    

    
    

In [ ]:
sns.catplot(x="Title", y="Age", hue='Sex', kind="swarm", data=all_data[all_data['Survived'].notnull()]);

In [ ]:
age_predict_mean = all_data.groupby(['Title','Sex']).Age.mean()

age_predicts = all_data[all_data.Age.isnull()]

for ind in age_predict_mean.index:
    age_predicts['Age'][(age_predicts['Title'] == ind[0])&(age_predicts['Sex'] == ind[1])] = age_predict_mean.iloc[0]

In [ ]:
age_known = all_data[all_data.Age.notnull()]

all_data = pd.concat([age_known,age_predicts], ignore_index=False) 

all_data.head()

In [ ]:
all_data.info()

Now all the features have no missing values except for `Survived`. The rows from `test_df` have no `Survived`. Next I will combine `SibSp` and `Parch` into a new feature as the family size. 

### SibSp & parch

I will first plot the relations to see if it is reasonable to combine these two features. 

In [ ]:
# Create a feature called the family size to see if this is relavant 
all_data['FamSiz'] = all_data['Parch'] + all_data['SibSp'] + 1

In [ ]:
sns.catplot(x="SibSp", y="Parch", hue='Survived', kind="swarm", data=all_data);

In [ ]:
sns.catplot(x="Survived", y="FamSiz", kind="swarm", data=all_data);

The plot on the top does not show obvious groups of `Parch` or `SibSp` with survival. But the bottom plot shows that if you are a family with more than 7 people, judging from the plot, the chance of living is zero. Hence I can create a feature called `FamSiz_7` as familysize=7 

In [ ]:
all_data['FamSiz_7'] = (all_data['FamSiz'] > 6).astype(int)

In [ ]:
all_data.drop(['FamSiz'], axis=1, inplace=True)

all_data.drop(['SibSp','Parch'],axis=1,inplace=True)

all_data.tail()

In [ ]:
all_data.info()

## Clean up the data (get dummies)

In [ ]:
# create dummy variables for categorical features
pclass_dummies = pd.get_dummies(all_data.Pclass, prefix="Pclass")
title_dummies = pd.get_dummies(all_data.Title, prefix="Title")
cabin_dummies = pd.get_dummies(all_data.Cabin, prefix="Cabin")
embarked_dummies = pd.get_dummies(all_data.Embarked, prefix="Embarked")

# concatenate dummy columns with main dataset
all_data_dummies = pd.concat([all_data, pclass_dummies, title_dummies, cabin_dummies, embarked_dummies], axis=1)

# drop categorical fields
all_data_dummies.drop(['Pclass', 'Title', 'Cabin', 'Embarked'], axis=1, inplace=True)

all_data_dummies.columns

In [ ]:
all_data_dummies.info()

# Model training

For the model training, I adopted four different methods
    - decision tree
    - kNN
    - random forest
    - logstic regression 

The following codes are based applications of these four methods. 

In [ ]:
# make sure the train data and test data are correct in length 
print(all_data.Survived.notnull().sum())
print(all_data.Survived.isnull().sum())

In [ ]:
train_data = all_data[all_data.Survived.notnull()]
test_data = all_data[all_data.Survived.isnull()]
y = train_data['Survived']
train_data.drop(['Survived'],axis=1,inplace=True)
test_data.drop(['Survived'],axis=1,inplace=True)

In [ ]:
test_data.head()

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression


from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, cross_val_score
scaler = StandardScaler()
scaler

In [ ]:
## create train data and dev data 
X_train, X_holdout, y_train, y_holdout = train_test_split(train_data.values, y, test_size=0.3,
random_state=17)

## Decision tree

In [ ]:
tree = DecisionTreeClassifier(max_depth=5, random_state=17)


tree_params = {'max_depth': range(2,7),
               'max_features': range(2,15)}

tree_grid = GridSearchCV(tree, tree_params,
                         cv=5, n_jobs=-1, verbose=True) 

tree_grid.fit(X_train, y_train)

## kNN

In [ ]:
# for kNN, we need to scale features
knn = KNeighborsClassifier(n_neighbors=7)
scaler = StandardScaler()  
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)
knn_pipe = Pipeline([('scaler', StandardScaler()), ('knn', KNeighborsClassifier(n_jobs=-1))])
knn_params = {'knn__n_neighbors': range(1, 10)}
knn_grid = GridSearchCV(knn_pipe, knn_params, cv=5, n_jobs=-1, verbose=True)
knn_grid.fit(X_train, y_train)

## Random forest

In [ ]:
forest = RandomForestClassifier(n_estimators=100, n_jobs=-1, random_state=17)
forest_params = {'max_depth': range(2,12),
                 'max_features': range(2,12)}

forest_grid = GridSearchCV(forest, forest_params,
                           cv=5, n_jobs=-1, verbose=True)

forest_grid.fit(X_train, y_train)

## Logistic regression 

In [ ]:
grid={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}# l1 lasso l2 ridge
logreg=LogisticRegression()
logreg_grid=GridSearchCV(logreg,grid,cv=10)
logreg_grid.fit(X_train,y_train)

## Determine the best method

The most appropriate method is the one with the highest dev score 

In [ ]:
dev_score = [accuracy_score(y_holdout, tree_grid.predict(X_holdout)),
             accuracy_score(y_holdout, knn_grid.predict(X_holdout)),
             accuracy_score(y_holdout, forest_grid.predict(X_holdout)),
             accuracy_score(y_holdout, logreg_grid.predict(X_holdout)),]

In [ ]:
print('Decision Tree:')
# print('best parameters: ',tree_grid.best_params_) #{'max_depth': 6, 'max_features': 17}
print('best score: ',tree_grid.best_score_ )
print('Dev score: ',accuracy_score(y_holdout, tree_grid.predict(X_holdout)))
print('kNN:')
# print('best parameters: ',knn_grid.best_params_) #{'max_depth': 6, 'max_features': 17}
print('best score: ',knn_grid.best_score_ )
print('Dev score: ',accuracy_score(y_holdout, knn_grid.predict(X_holdout)))
print('Random Forest:')
# print('best parameters: ',forest_grid.best_params_) #{'max_depth': 6, 'max_features': 17}
print('best score: ',forest_grid.best_score_ )
print('Dev score: ',accuracy_score(y_holdout, forest_grid.predict(X_holdout)))
print('Logistic Regression:')
# print('best parameters: ',forest_grid.best_params_) #{'max_depth': 6, 'max_features': 17}
print('best score: ',logreg_grid.best_score_ )
print('Dev score: ',accuracy_score(y_holdout, logreg_grid.predict(X_holdout)))

Here we can see that decision tree gives the highest score. So I will use this model for prediction. 

## Prediction

In [ ]:
test_survive = tree_grid.predict(test_data)
test_data['Survive'] = test_survive

In [ ]:
test_data = test_data.sort_index()

In [ ]:
output = pd.DataFrame({'PassengerId': PassengerId, 'Survived': test_data['Survive'].astype(int)})
output.to_csv('my_submission_v9.csv', index=False)
print("Your submission was successfully saved!")

This prediction gives 78% accuracy. 